# Transient one-dimensional flow - hydraulic head response to stream flood

If an aquifer hydraulically interacts with a river or stream, surface water floods will propagate into the aquifer. In the following, we look at a semi-infinite confined aquifer, i.e. a confined aquifer that interacts with a fully penetrating river on one side and has no boundary on the other side (Figure 1). This situation can be described by the following one-dimensional groundwater flow equation (assuming a homogeneous aquifer without sinks or sources, e.g. no recharge from precipitation):

$$\frac{\partial² h}{\partial x²} = \frac {S \partial h}{T \partial t}$$

where $h$ is hydraulic head, $x$ is distance from the surface water, $S$ is storage coefficient (storativity), $T$ is transmissivity, and $t$ is time.
We assume the aquifer is initially in equlibrium with the surface water at $h = 0$, i.e. the hydraulic heads of the aquifer are initally equal to the surface water level, which is described by the following initial condition:

$$h(x,t=0)=0$$ 

At $t = 0$ the surface water level and thus the hydraulic head at $x = 0$ is subject to a sudden rise to $h_0$, which is described by the boundary condition:

$$h(0,t>0) = h_0$$

The solution of the above flow equation for the given intial and boundary conditions is

$$h(x,t)=h_0 \: \mathrm{ erfc}\left(\sqrt{\frac{S x^2}{4T (t)}}\right)$$

This solution can also be applied to unconfined aquifers if the variation of the hydraulic head is small relative to the saturated thickness of the aquifer such that the transmissivity can be regarded as approximately constant. 

Bakker & Post (2022) provide more details about the derivation of this solution and a Python code for its application. This Jupyter Notebook makes use of their Python code and adds features such as sliders and textboxes that facilitate the variation of parameter values and the visualisation of their effects on the hydraulic heads.
<br><br>

<img src='FIGS\Fig1_transient_1d_flow_conf.jpg' width="400"> <br>
*Figure 1: Aquifer headchange due to surface water level change.*


References: Bakker, M., & Post, V. (2022). Analytical groundwater modeling: Theory and applications using Python. CRC Press.

In [29]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from ipywidgets import *
from scipy.special import erfc
from datetime import datetime

In [ ]:
# Parameters
T = widgets.IntSlider(value=10, min=0, max=100, step=0.1, layout=Layout(width='220px'))
S = widgets.FloatLogSlider(value=10e-1, base=10, min=-6, max=-0.52, layout=Layout(width='220px')) 
h0 = widgets.FloatSlider(value=2, min=0.1, max=10, step=0.1, layout=Layout(width='220px')) # change in river level, m
max_x = widgets.IntText(value=200, min=1, max=1000, step=10, layout=Layout(width='220px')) # maximal plottet distance from the river
max_t = widgets.IntText(value=100, min=1, max=1000, step=10, layout=Layout(width='220px')) # maximal plottet distance from the river
t0 = 0 # time of change in river level, d

# Initialize to save previously plotted lines and their parameters
saved_lines = []

In [31]:
def h_edelman(x, t, T, S, h0=1, t0=0): 
    # Funktion to evaluate the head change after Edelman (1947)
    u = np.sqrt(S * x ** 2 / (4 * T * (t - t0)))
    return h0 * erfc(u)

def Qx_edelman(x, t, T, S, h0, t0=0):  
    # Funktion to evaluate the 1d flux after Darcy(1856) and Edelman (1947)
    u = np.sqrt(S * x ** 2 / (4 * T * (t - t0)))
    return T * h0 * 2 * u / (x * np.sqrt(np.pi)) * np.exp(-u ** 2)


In [32]:
def plot_sol(T, S, h0, max_x, max_t, t0, replot_saved=True):
    global fig, ax1, ax2, ax3, ax4
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 15))
    plt.subplots_adjust(hspace=1,bottom=0.25)
    x = np.linspace(1e-12, max_x, 100)
    t = np.linspace(1e-12, max_t, 100)
    

    # Plot saved lines with decreasing alpha
    if replot_saved:
        base_alpha = 1.0  # Start with full opacity for the most recent saved line
        alpha_decrement = 0.4  # Adjust this as needed for fading effect
        for i, line in enumerate(reversed(saved_lines)):
            alpha = max(0.1, base_alpha - i * alpha_decrement)  # Ensure alpha stays above 0.1 for visibility
            plot_saved_lines(line, x, t, alpha=alpha)
    
    for time, c in zip([1, 10, 100], ['limegreen', 'dodgerblue', 'mediumslateblue']):    
        h = h_edelman(x, time, T, S, h0, t0)
        ax1.plot(x, h, label=f'time={time} d', color=c) 
    
    ax1.grid()
    ax1.set_xlabel('$x$ [m]')
    ax1.set_ylabel('h [m]')
    ax1.set_ylim(0, h0)
    ax1.set_xlim(0, max_x)

    for time, c in zip([1, 10, 100],  ['limegreen', 'dodgerblue', 'blueviolet']):  
        Qx = Qx_edelman(x, time, T, S, h0, t0)
        ax2.plot(x, Qx, label=f'time={time} d', color=c)  
    ax2.grid()
    ax2.set_xlabel('$x$ [m]')
    ax2.set_ylabel('$Q_x$ [m$^2$/d]')
    ax2.set_xlim(0, max_x)

    for dist, c in zip([50, 100, 200], ['firebrick', 'darkorange', 'gold']):   
        h = h_edelman(dist, t, T, S, h0, t0)
        ax3.plot(t, h, label=f'distance={dist} m', color=c) 
    ax3.grid()
    ax3.set_xlabel('$t$ [d]')
    ax3.set_ylabel('h [m]')
    ax3.set_ylim(0, h0)
    ax3.set_xlim(0, max_t)

    for dist, c in zip([50, 100, 200],  ['firebrick', 'darkorange', 'gold']):   
        Qx = Qx_edelman(dist, t, T, S, h0, t0)
        ax4.plot(t, Qx, label=f'distance={dist} d', color=c)
    ax4.grid()
    ax4.set_xlabel('$t$ [d]')
    ax4.set_ylabel('$Q_x$ [m$^2$/d]')
    ax4.set_xlim(0, max_t)

    # Legends
    global lgt1, lgt2
    handles_time, labels_time = ax1.get_legend_handles_labels()  # Collect from ax1 and ax2
    lgt1=fig.legend(handles_time, labels_time, loc='upper center', bbox_to_anchor=(0.5, 0.6), ncol=3)  # Custom position

    handles_distance, labels_distance = ax3.get_legend_handles_labels()
    lgt2=fig.legend(handles_distance, labels_distance, loc='lower center', bbox_to_anchor=(0.5, 0.1), ncol=3)

    # Plot titles
    #fig.text(0.5, 0.90, '...as a funktion of distance', ha='center', va='center', fontsize=16, fontweight='bold')
    #fig.text(0.5, 0.48, '... as a funktion of time', ha='center', va='center', fontsize=16, fontweight='bold')

In [33]:

def plot_saved_lines(line, x, t, alpha=1.0):
    # Retrieve the saved parameters for each line
    T_saved = line['T']
    S_saved = line['S']
    h0_saved = line['delta_h']
    
    # Fixed colors and line styles
    colors1 = ['limegreen', 'dodgerblue', 'blueviolet']
    colors2 = ['firebrick', 'darkorange', 'gold']
    linestyles = ['--', '-.', ':']

    # Plot saved head changes with explicit color, style, and transparency
    for i, time in enumerate([1, 10, 100]):
        h = h_edelman(x, time, T_saved, S_saved, h0_saved)
        ax1.plot(x, h, linestyle=linestyles[i % len(linestyles)], color=colors1[i % len(colors1)], 
                 label=f'saved time={time}, T={T_saved}, S={S_saved:.2f}', linewidth=2, alpha=alpha)

        Qx = Qx_edelman(x, time, T_saved, S_saved, h0_saved)
        ax2.plot(x, Qx, linestyle=linestyles[i % len(linestyles)], color=colors1[i % len(colors1)], 
                 label=f'saved time={time}, T={T_saved}, S={S_saved:.2f}', linewidth=2, alpha=alpha)

    for i, dist in enumerate([50, 100, 200]):
        h = h_edelman(dist, t, T_saved, S_saved, h0_saved)
        ax3.plot(t, h, linestyle=linestyles[i % len(linestyles)], color=colors2[i % len(colors2)], 
                 label=f'saved dist={dist}, T={T_saved}, S={S_saved:.2f}', linewidth=2, alpha=alpha)

        Qx = Qx_edelman(dist, t, T_saved, S_saved, h0_saved)
        ax4.plot(t, Qx, linestyle=linestyles[i % len(linestyles)], color=colors2[i % len(colors2)], 
                 label=f'saved dist={dist}, T={T_saved}, S={S_saved:.2f}', linewidth=2, alpha=alpha)


In [34]:
def on_button_clicked(b):
    global saved_lines
    saved_lines.append({
        'T': T.value,
        'S': S.value,
        'delta_h': h0.value,
        'max_x': max_x.value,
        'max_t': max_t.value,
        'alpha': 1.0  # Full opacity for new saves
    })

def on_sfbutton_clicked(b):
    # Function to handle button click event

    now= datetime.now()
    dt_strg= now.strftime('%H%M%S_%Y%m%d_')
    fig.savefig(dt_strg+'transient_1d_flow.png', bbox_extra_artists=(lgt1,lgt2,), bbox_inches='tight')
    

In [35]:

# Connect the button click event to the function
button = widgets.Button(description='save lines')   # Button for saving current lines
button.on_click(on_button_clicked)

sfbutton = widgets.Button(description='save figure')
sfbutton.on_click(on_sfbutton_clicked)

# Output layout
out = widgets.interactive_output(plot_sol, {'T':T, 'S':S, 'h0':h0, 'max_x':max_x, 'max_t':max_t, 't0':widgets.fixed(t0)})
grid = GridspecLayout(15, 8)

# Plot
grid[:1, :] = HTML('<h1> Groundwater response to abrupt change in surface water level </h1>')
grid[1:, 2:] = out

# Input widgets and buttons
grid[1:2, :2] = VBox([HTML('T [m²/d] <br> Transmissivity'), T])
grid[2:3, :2] = VBox([HTML('h0 [m] <br> Water level to which the river rises <br> from an initial level of zero'), h0])
grid[3:4, :2] = VBox([HTML('S [-] <br> Storativity (storage coefficient)'), S])
grid[4:5, :2] = VBox([HTML('max_x [m] <br> (Maximal) distance from the river'), max_x])
grid[5:6, :2] = VBox([HTML('max_t [m] <br> (Maximal) time since sudden rise <br> in the water level of the river'), max_t])

grid[7:8, :2] = HBox([button, sfbutton]) 



grid

GridspecLayout(children=(HTML(value='<h1> Groundwater response to abrupt change in surface water level </h1>',…

This work &copy; 2024 by Edith Grießer, Steffen Birk (University of Graz) is licensed under  <a href="https://creativecommons.org/licenses/by/4.0/?ref=chooser-v1" target="_blank" rel="license noopener noreferrer" style="display:inline-block;">CC BY 4.0<img style="height:22px!important;margin-left:3px;vertical-align:text-bottom;" src="https://mirrors.creativecommons.org/presskit/icons/cc.svg?ref=chooser-v1" alt=""><img style="height:22px!important;margin-left:3px;vertical-align:text-bottom;" src="https://mirrors.creativecommons.org/presskit/icons/by.svg?ref=chooser-v1" alt=""></a></p> 